In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# TODO

(х) очистить возраст - максимальные выбросы(заменить на медиану), 1-17 лет (если опыт 0-50 лет, возраст=опыт+18), пустые()

(х) очистить опыт - опыт больше возраста(заменить на средний опыт по возрасту)

(х) очистить должность - нижний регистр без пунктуации

## read csv

In [2]:
#read and print train data
raw_train_data = pd.read_csv('train.csv', sep=';')

In [24]:
raw_train_data[['age','experience','position']].head()

,age,experience,position
0,21.0,3,Специалист
1,26.0,4,Лаборант
2,36.0,12,"Специалист, администратор, бухгалтер, экономист"
3,34.0,9,Специалист
4,25.0,1,Инструктор по плаванию


In [25]:
raw_train_data[['age','experience','position']].describe(include='all')

,age,experience,position
count,298459.000000,306270.000000,306270
unique,NaN,NaN,77762
top,NaN,NaN,Специалист
freq,NaN,NaN,10098
mean,35.661344,5.748225,NaN
std,12.616996,7.745237,NaN
min,2.000000,0.000000,NaN
25%,28.000000,0.000000,NaN
50%,34.000000,3.000000,NaN
75%,42.000000,9.000000,NaN


In [27]:
raw_train_data['experience'].mean()

5.748225422013256

In [30]:
raw_train_data[raw_train_data['experience'] > raw_train_data['age']]['experience'].unique()

array([ 49, 108,  65,  66, 102,  51,  55,  59, 122,  46,  43,  52,  97,
         8,  40,  56, 902,  86,  62, 154,   7,  99,  64, 937, 100],
      dtype=int64)

In [31]:
raw_train_data[raw_train_data['experience'] > raw_train_data['age']]['experience'].count()

27

## clean age column

In [9]:
raw_train_data['age'].unique()

array([  21.,   26.,   36.,   34.,   25.,   49.,   nan,   28.,   32.,
         23.,   38.,   41.,   53.,   27.,   37.,   31.,   33.,   54.,
         44.,   55.,   57.,   42.,   40.,   29.,   30.,   60.,   45.,
         39.,   19.,   22.,   56.,   58.,   52.,   35.,   46.,   24.,
         43.,   59.,   16.,   18.,   50.,   51.,   47.,   20.,   48.,
         70.,   61.,   17.,   68.,   63.,   66.,   72.,   15.,   64.,
         62.,   67.,   65.,   69.,   79.,   73.,   71.,    5.,    7.,
          4.,   75.,  132.,   77.,  119.,   74.,   14.,   76., 1933.,
       1928.,    2., 1926., 1936.,    8., 1924.,    3.,   82.])

In [10]:
raw_train_data['age'].isna().sum()

7811

### max outliers

In [11]:
mask = (raw_train_data['age'] > 100.0)
raw_train_data['age_cleaned'] = np.where(mask, round(raw_train_data['age'].median(),1), raw_train_data['age'])

In [12]:
raw_train_data['age_cleaned'].unique()

array([21., 26., 36., 34., 25., 49., nan, 28., 32., 23., 38., 41., 53.,
       27., 37., 31., 33., 54., 44., 55., 57., 42., 40., 29., 30., 60.,
       45., 39., 19., 22., 56., 58., 52., 35., 46., 24., 43., 59., 16.,
       18., 50., 51., 47., 20., 48., 70., 61., 17., 68., 63., 66., 72.,
       15., 64., 62., 67., 65., 69., 79., 73., 71.,  5.,  7.,  4., 75.,
       77., 74., 14., 76.,  2.,  8.,  3., 82.])

In [13]:
masks = [(raw_train_data['age'] < 18.0),(raw_train_data['age'].isna()),(raw_train_data['experience'] > 50)]
mask = [(a or b) and c for a,b,c in zip(*masks)]

raw_train_data['age_cleaned'] = np.where(mask, round(raw_train_data['age'].median(),1), raw_train_data['age_cleaned'])
raw_train_data['age_cleaned'].unique()

array([21., 26., 36., 34., 25., 49., nan, 28., 32., 23., 38., 41., 53.,
       27., 37., 31., 33., 54., 44., 55., 57., 42., 40., 29., 30., 60.,
       45., 39., 19., 22., 56., 58., 52., 35., 46., 24., 43., 59., 16.,
       18., 50., 51., 47., 20., 48., 70., 61., 17., 68., 63., 66., 72.,
       15., 64., 62., 67., 65., 69., 79., 73., 71.,  5.,  7.,  4., 75.,
       77., 74., 14., 76.,  2.,  8.,  3., 82.])

### 1-17 y.o.

In [14]:
masks = [(raw_train_data['age'].isna()),(raw_train_data['age'] < 18.0),(raw_train_data['experience'] >= 0),(raw_train_data['experience'] <= 50)]
mask = [(a or b) and c and d for a,b,c,d in zip(*masks)]

raw_train_data['age_cleaned'] = np.where(mask, raw_train_data['experience']+18.0, raw_train_data['age_cleaned'])
raw_train_data['age_cleaned'].unique()

array([21., 26., 36., 34., 25., 49., 35., 28., 32., 23., 38., 41., 53.,
       27., 37., 31., 33., 54., 44., 55., 57., 42., 40., 29., 30., 60.,
       45., 39., 19., 22., 56., 18., 58., 52., 46., 24., 43., 59., 50.,
       51., 47., 20., 48., 70., 61., 68., 63., 66., 72., 64., 62., 67.,
       65., 69., 79., 73., 71., 75., 77., 74., 76., 82.])

In [15]:
raw_train_data['age_cleaned'].isna().sum()

0

# clean experience column

In [16]:
mapping = raw_train_data[raw_train_data['age_cleaned']>raw_train_data['experience']].groupby('age_cleaned').mean().apply(list).to_dict()['experience']

In [17]:
raw_train_data['experience_cleaned'] = raw_train_data['experience']
raw_train_data.loc[raw_train_data['age_cleaned']<raw_train_data['experience_cleaned'],'experience_cleaned'] = raw_train_data[raw_train_data['age_cleaned']<raw_train_data['experience_cleaned']]['age_cleaned'].map(mapping)

# preprocess position

In [18]:
import string

In [19]:
raw_train_data['position_cleaned'] = raw_train_data['position'].str.lower()

In [20]:
raw_train_data['position_cleaned'] = raw_train_data['position_cleaned'].str.translate(str.maketrans('', '', string.punctuation))

In [32]:
raw_train_data[['age_cleaned','experience_cleaned','position']].describe(include='all')

,age_cleaned,experience_cleaned,position
count,306270.000000,306270.000000,306270
unique,NaN,NaN,77762
top,NaN,NaN,Специалист
freq,NaN,NaN,10098
mean,35.278581,5.736430,NaN
std,10.110358,7.350172,NaN
min,18.000000,0.000000,NaN
25%,27.000000,0.000000,NaN
50%,34.000000,3.000000,NaN
75%,42.000000,9.000000,NaN
